# Merge tracts into neighborhoods from LA county

By: Sandhya Kambhampati 

We want to be able to combine tracts for analysis. In order to do this, we will need a lookup for tracts to neighborhoods. This notebook will create a lookup that can be used for ACS Census data that is on a tract-level.

In [ ]:
import warnings, os
import pandas as pd
import numpy as np
import geopandas as gpd
%matplotlib inline

In [ ]:
# load in the LA county tracts file 
tracts = gpd.read_file("../data/input/lac-tracts.geojson")

In [ ]:
tracts.plot()

In [ ]:
tracts.head()

In [ ]:
len(tracts)

In [ ]:
trimmed_tracts = tracts[[
    'TRACTCE',
    'geometry'
]].rename(columns={"TRACTCE": "tract"})

In [ ]:
neighborhood = gpd.read_file("../data/input/la_county.json")

In [ ]:
neighborhood.head(1)

In [ ]:
neighborhood.plot()

In [ ]:
trimmed_neighborhoods = neighborhood[[
    'name',
    'geometry'
]].rename(columns={"name": "neighborhood"})

In [ ]:
trimmed_neighborhoods = trimmed_neighborhoods.to_crs("epsg:2229")

In [ ]:
trimmed_tracts = trimmed_tracts.to_crs("epsg:2229")

In [ ]:
# smash the two shapefiles together
trimmed_neighborhoods_tracts = gpd.overlay(trimmed_tracts,trimmed_neighborhoods, how='intersection')

In [ ]:
trimmed_neighborhoods_tracts.plot()

In [ ]:
len(trimmed_neighborhoods_tracts)

In [ ]:
trimmed_neighborhoods_tracts = trimmed_neighborhoods_tracts.to_crs(2229)

In [ ]:
# calculate the area covered 
trimmed_neighborhoods_tracts['area'] = trimmed_neighborhoods_tracts['geometry'].area

In [ ]:
trimmed_neighborhoods_tracts.sort_values('tract').head()

In [ ]:
trimmed_tracts = trimmed_tracts.to_crs(2229)

In [ ]:
# calculate the tracts covered 
trimmed_tracts['total_tract_area'] = trimmed_tracts['geometry'].area

In [ ]:
# calculations
calcs = trimmed_neighborhoods_tracts.drop('geometry', axis=1)\
.merge(trimmed_tracts.drop('geometry', axis=1))\
.sort_values('tract')

In [ ]:
# tract percent 
calcs['tract_percent'] = calcs['area']/calcs['total_tract_area']

In [ ]:
len(calcs)

In [ ]:
len(trimmed_neighborhoods_tracts)

In [ ]:
calcs.head()

In [ ]:
calcs.tract.nunique()

In [ ]:
# export the csv
calcs.to_csv('../data/output/neighborhood/la_tract_to_neighborhood_portions.csv', index=False)